# Aerhan Srirangan's Final Project

# Imports

In [1]:
# necessary imports
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import datetime as dt

from keras.layers import *
from keras.models import Sequential
from keras.utils import plot_model
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Specify tensorflow version
print("Tensorflow Version:", tf.__version__)

2023-08-02 01:27:43.661381: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Tensorflow Version: 2.10.0


In [2]:
# Specify current directory
print("Current directory: ", os.getcwd())
# List current directory
print("Directory List", os.listdir(os.getcwd()))
# Specify the directory containing the dataset
DATASET_DIR = "UCF-Crime Dataset"
# Specify path to dataset directory
dataset_path = os.path.join(os.getcwd(), DATASET_DIR)
# Get each category of the dataset
dirs = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
print(dirs)
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64

Current directory:  /Users/aerhansrirangan/Documents/Uni/Year 3/Final Year Project/Model
Directory List ['~$nal Implementation Report.docx', 'UCF-Crime Dataset', 'Final Project.py', '.DS_Store', 'Final Implementation Report.pdf', 'Final_Year_Project.ipynb', 'Final Implementation Report.docx', 'convlstm_model_structure_plot.png', '.ipynb_checkpoints', 'Final Project.ipynb', '.idea']
['Arson', 'Shoplifting', 'Stealing', 'Burglary', 'Road Accidents', 'Fighting', 'Vandalism', 'Explosion', 'Arrest', 'Normal', 'Abuse', 'Robbery', 'Assault', 'Shooting']


# Fetch Dataset

## Video class

In [3]:
class Video:
    def __init__(self, name: str, frames, filepath: str, label: str, frame_count:int):
        self.name = name
        self.frames = frames
        self.filepath = filepath
        self.label = label
        self.frame_count = frame_count
    
    

### Video Length function

In [4]:
# helper function to display the length of the video
def getVideoDuration(frame_count: float, fps: float) -> str:
    """
    Calculates duration of the video in hh:mm:ss format
    :param frame_count: number of frames in the video
    :param fps: frame rate of the video
    :return: a string representing the duration of the video in hh:mm:ss format
    """
    # Calculate the length of the video in seconds
    length = frame_count / fps

    # Calculate the duration in hours, minutes, and seconds
    hours, remainder = divmod(length, 3600)
    minutes, seconds = divmod(remainder, 60)

    # Return the duration as a formatted string
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}"


As the dataset is 37GB, I will only use a tiny portion of it (10 videos per label). Otherwise, it will be too extensive to compute.

In [5]:
# extracting frames from video
def extractFrames(video):
    # A list to store the frames of the video
    frames = []
    while video.isOpened():
        # Read the current frame
        ret, frame = video.read()
        # If the frame was not successfully read, break out of the loop
        if not ret:
            break
        # Resize the Frame to fixed height and width.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        # Normalize the resized frame by dividing it with 255 so each pixel value then lies between 0 and 1
        normalised_frame = resized_frame / 255
        frames.append(normalised_frame)
    video.release()
    return frames

In [6]:
# to make the video length the same, I will split the videos into equal length 
def splitVideo(video, interval):
    """
    Splits a video into separate parts given the interval of frames.
    Args:
        video: The video as an array.
        interval: The interval of frames.
    Returns:
        A list of the split videos.
  """
    print(len(video))
    split_videos = []
    start = 0
    while start < len(video):
        end = min(start + interval, len(video))
        print(f"start = {start}, end  = {end}")
        split_videos.append(video[start:end])
        print(len(split_videos))
        start = end + 1
    return split_videos


In [7]:
# function to fetch dataset stored on hard drive as it's 37GB
# and checks if the videos in the dataset are usable or not
def fetchDataset():
    videos = []

    for label in dirs:
        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of: {label}')
        label_path = os.path.join(dataset_path, label)
        print("Label path:", label_path)
        files = [f for f in os.listdir(label_path) if f.endswith('.mp4')]
        # Select the first 10 videos of each label
        file_count = 0
        for file in files:
            filepath = os.path.join(label_path, file)
            # Check if the video file can be opened
            video = cv2.VideoCapture(filepath)
            if file_count == 10:
                break
            if not video.isOpened():
                print(f"Could not open video file: {filepath}")
                files.remove(file)
                continue
            else:
                fps = video.get(cv2.CAP_PROP_FPS)
                frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
                duration = getVideoDuration(frame_count, fps)
                print(file, "has ", duration)            
                # Extract the frames of the video
                frames = extractFrames(video)
                
                video = Video(file, frames, filepath, label, frame_count)
                # Add video to videoarray
                videos.append(video)
                
                file_count += 1
    return videos

In [ ]:
videos = fetchDataset()

print("Number of videos", len(videos))
frame_counts = [int(video.frame_count) for video in videos]
min_frames = min(frame_counts)
max_frames = max(frame_counts)
print("max number of frames in all the videos", max_frames)
print("min number of frames in all the videos", min_frames)

# As the videos are of varying length, I have decided to split the videos into parts  


for i in range(len(videos)):
    if (videos[i].frame_count > min_frames):
        video_parts = splitVideo(videos[i].frames, min_frames)
        print(len(video_parts))
        if (len(video_parts) > 1):
            for j in range(len(video_parts)):
                video_name = str(str(videos[i].name) +" Part " + str(j+1))
                new_video = Video(video_name, video_parts[j], videos[i].filepath, videos[i].label, 
                                  len(video_parts[j]))
                videos.insert(i + j + 1, new_video)
            videos.pop(i)
video_labels = [video.label for video in videos]
features = [video.frames for video in videos]
print(f"features has length of {len(features)}")
print(f"Video Labels: {video_labels} has length of {len(video_labels)}")

Extracting Data of: Arson
Label path: /Users/aerhansrirangan/Documents/Uni/Year 3/Final Year Project/Model/UCF-Crime Dataset/Arson


[ WARN:0@1.683] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ERROR:0@1.683] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/videoio/src/cap.cpp (166) open VIDEOIO(GSTREAMER): raised OpenCV exception:

OpenCV(4.6.0) /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/core/src/utils/filesystem.cpp:580: error: (-213:The function/feature is not implemented)  in function 'exists'




Arson014_x264.mp4 has  00:01:28


[ WARN:0@3.795] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ERROR:0@3.795] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/videoio/src/cap.cpp (166) open VIDEOIO(GSTREAMER): raised OpenCV exception:

OpenCV(4.6.0) /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/core/src/utils/filesystem.cpp:580: error: (-213:The function/feature is not implemented)  in function 'exists'




Arson015_x264.mp4 has  00:05:41


...

Fighting041_x264.mp4 has  00:17:34


[ WARN:0@274.993] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ERROR:0@274.993] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/videoio/src/cap.cpp (166) open VIDEOIO(GSTREAMER): raised OpenCV exception:

OpenCV(4.6.0) /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/core/src/utils/filesystem.cpp:580: error: (-213:The function/feature is not implemented)  in function 'exists'




Fighting040_x264.mp4 has  00:01:17


[ WARN:0@278.069] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ERROR:0@278.069] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/videoio/src/cap.cpp (166) open VIDEOIO(GSTREAMER): raised OpenCV exception:

OpenCV(4.6.0) /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/core/src/utils/filesystem.cpp:580: error: (-213:The function/feature is not implemented)  in function 'exists'




Fighting008_x264.mp4 has  00:07:17


# Create Test and Train Set

In [ ]:
# # splitting into train and test sets
# video_labels = np.array(video_labels)
# features_train, features_test, labels_train, labels_test = train_test_split(features, 
#                                                                             video_labels,
#                                                                             test_size=0.2, shuffle=True)
# print(labels_train)
# print(features_train)

# Encoding labels

I'll be using One Hot Encoding as it is more flexible and since there is no natural order or hierarchy

In [ ]:
# # Convert labels to numerical categories
# print("Numpy array labels:", labels_train)
# encoder = OneHotEncoder()
# # Reshape the array to 2D
# labels_train = labels_train.reshape(-1, 1)
# labels_train = np.array(labels_train)
# features_train = np.array(features_train)
# # Create an instance of the OneHotEncoder
# encoder = OneHotEncoder()

# # Fit the encoder to the data
# encoder.fit(labels_train)

# # Transform the data
# one_hot_encoded_labels = encoder.transform(labels_train)
# print("One hot encoded labels", one_hot_encoded_labels)

## Create ConvLSTM model

In [ ]:
# def create_ConvLSTM_model():
#     model = Sequential()

#     model.add(ConvLSTM2D(filters=4, kernel_size=(3, 3), activation='tanh', data_format="channels_last",
#                          return_sequences=True, input_shape=(min_frames, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
#     model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
#     model.add(TimeDistributed(Dropout(0.05)))

#     model.add(ConvLSTM2D(filters=8, kernel_size=(3, 3), activation='tanh', data_format="channels_last",
#                          return_sequences=True))
#     model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
#     model.add(TimeDistributed(Dropout(0.05)))

#     model.add(ConvLSTM2D(filters=14, kernel_size=(3, 3), activation='tanh', data_format="channels_last",
#                          return_sequences=True))
#     model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
#     model.add(TimeDistributed(Dropout(0.05)))

#     model.add(ConvLSTM2D(filters=16, kernel_size=(3, 3), activation='tanh', data_format="channels_last",
#                          return_sequences=True))
#     model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))

#     model.add(Flatten())

#     model.add(Dense(len(dirs), activation="softmax"))

#     # Display the models summary.
#     model.summary()

#     # Return the constructed convlstm model.
#     return model

In [ ]:
# # Construct the required convlstm model.
# convlstm_model = create_ConvLSTM_model()
# # Display the success message.
# print("Model Created Successfully!")
# # Plot the structure of the constructed model.
# plot_model(convlstm_model, to_file='convlstm_model_structure_plot.png', show_shapes=True, show_layer_names=True)

# early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)


# # Compile the model and specify loss function, optimizer and metrics values to the model
# convlstm_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])

## Training Model

In [ ]:
# # Start training the model.
# convlstm_model_training_history = convlstm_model.fit(features_train, labels_train, epochs=50,
#                                                      batch_size=4, shuffle=True, validation_split=0.2,
#                                                      callbacks=[early_stopping_callback])
# # Evaluate the trained model.
# model_evaluation_history = convlstm_model.evaluate(features_test, labels_test)

# # Get the loss and accuracy from model_evaluation_history.
# model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history
 
# # Define the string date format.
# # Get the current Date and Time in a DateTime Object.
# # Convert the DateTime object to string according to the style mentioned in date_time_format string.
# date_time_format = '%Y_%m_%d__%H_%M_%S'
# current_date_time_dt = dt.datetime.now()
# current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)
 
# # Define a useful name for our model to make it easy for us while navigating through multiple saved models.
# model_file_name = f'convlstm_model___Date_Time_' \
#                   f'{current_date_time_string}___Loss_' \
#                   f'{model_evaluation_loss}___Accuracy_{model_evaluation_accuracy}.h5'
 
# # Save your Model.
# convlstm_model.save(model_file_name)

## Plots

In [ ]:
# def plot_metric(model_training_history, metric_name_1, metric_name_2, plot_name):
#     """
#     This function will plot the metrics passed to it in a graph.
#     Args:
#         model_training_history: A history object containing a record of training and validation
#                                 loss values and metrics values at successive epochs
#         metric_name_1:          The name of the first metric that needs to be plotted in the graph.
#         metric_name_2:          The name of the second metric that needs to be plotted in the graph.
#         plot_name:              The title of the graph.
#     """
    
#     # Get metric values using metric names as identifiers.
#     metric_value_1 = model_training_history.history[metric_name_1]
#     metric_value_2 = model_training_history.history[metric_name_2]
    
#     # Construct a range object which will be used as x-axis (horizontal plane) of the graph.
#     epochs = range(len(metric_value_1))
 
#     # Plot the Graph.
#     plt.plot(epochs, metric_value_1, 'blue', label=metric_name_1)
#     plt.plot(epochs, metric_value_2, 'red', label=metric_name_2)
 
#     # Add title to the plot.
#     plt.title(str(plot_name))
 
#     # Add legend to the plot.
#     plt.legend()


# # Visualize the training and validation loss metrics.
# plot_metric(convlstm_model_training_history, 'loss', 'val_loss', 
#             'Total Loss vs Total Validation Loss')

# # Visualize the training and validation accuracy metrics.
# plot_metric(convlstm_model_training_history, 'accuracy', 'val_accuracy', 
#             'Total Accuracy vs Total Validation Accuracy')